# PyTorch Workflow

Let's explore an example PyTorch end-to-end worflow.

In [5]:
what_were_covering = {
    1: "data {prepare and load}",
    2: "build model",
    3: "fitting the model to data (training)",
    4: "making predictions",
    5: "saving and loading the model",
    6: "putting it all together"
}

what_were_covering

{1: 'data {prepare and load}',
 2: 'build model',
 3: 'fitting the model to data (training)',
 4: 'making predictions',
 5: 'saving and loading the model',
 6: 'putting it all together'}

In [7]:
import torch
from torch import nn ## nn contains all of PyTorch's building blocks for neural netowrks
import matplotlib.pyplot as plt

# check PyTorch version
torch.__version__

'1.13.1'

## Data (preparing and loading)

Data can be almost anything... in machine learning.

* Excel spreadsheet
* Images of nay kind
* Videos (YouTube has lots of data...)
* Audio like songs or podcasts
* DNA
* Text

Machine learning is a game of two parts:
1. Get data into a numerical representation.
2. Build a model to learn patterns in that numerical presentation.

To showcase this, let's create some *known* data using the linear regression formula (`y = mx + c -> where m is the slope and c is the y intercept`).

We'll use a linear regression formula to make s straight line with *known* **parameters**.

In [8]:
# create *known* parameters
weight = 0.7
bias = 0.3

# create data
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50)

In [9]:
len(X), len(y)

(50, 50)